In [ ]:
pip install langchain-experimental psutil gradio

# Import statements

In [ ]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
import psutil
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

# Connect to locally running LLM by using Ollama

In [2]:
llm = Ollama(model="llama3.1:8b-instruct-q3_K_S", num_thread = (psutil.cpu_count() - 3), keep_alive = -1, temperature=0.7, num_ctx=8000)
#llm.invoke("hi") #test

# Prompts using generate knowldge prompting and prompt chaining

In [3]:
generate_knowledge_prompt_Str = """
what are top 5 things to consider when writing {section} description 
for my resume. List them in bullet points and provide examples. Limit 10 lines.
"""
generate_knowledge_prompt = PromptTemplate(
    template = generate_knowledge_prompt_Str,
    input_variables = ["section"]
)


task_overview_prompt_str = """ 
I need help tailoring {section} section of my resume to better align with a specific JOB DESCRIPTION. 
The goal is to make sure the tailored {section} effectively showcases my relevant skills and experiences, 
increasing the likelihood of passing through automated screening processes like Applicant Tracking Systems (ATS) for given JOB DESCRIPTION.
"""
task_overview_prompt = PromptTemplate(
    template = task_overview_prompt_str,
    input_variables = ["section"]
)

input_values_prompt_str = """
Here are the details you'll need:

My resume {section} section: 
{section_content}


JOB DESCRIPTION:
{job_description}
"""
input_values_prompt = PromptTemplate(
    template = input_values_prompt_str,
    input_variables = ["section", "section_content", "job_description"] 
)

detail_instruction_prompt_str = """
When tailoring the resume {section} section to better align with the given JOB DESCRIPTION by showcasing my relevant skills and experiences:

	1.	Do Not Add: Don't introduce new technologies, skills, work experience, or education that are not already present.
	2.	Rewording/Summarization: Feel free to reword or summarize to highlight relevant experience, using action verbs and quantifiable results.
	3.	ATS Compatibility: Ensure the output is ATS-friendly, using standard section headings, avoiding complex formatting, and maintaining clear bullet points.
	4.	Focus on Keywords: Prioritize the use of keywords from the JOB DESCRIPTION that match my existing experience or skills. Ensure that these keywords are incorporated naturally and appropriately.

{generated_knowledge}
"""
detail_instruction_prompt = PromptTemplate(
    template = detail_instruction_prompt_str,
    input_variables = ["section", "generated_knowledge"] 
)

tailor_section_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are going to act as a professional resume writer skilled in presenting information concisely and using niche-appropriate language, while avoiding redundancy and cliché terms."),
    MessagesPlaceholder(variable_name="messages")
    ])

# print(task_overview_prompt.invoke({"section":""}).text) # test

# Create chains

In [4]:
generate_knowledge_chain = generate_knowledge_prompt | llm

tailor_section_chain = tailor_section_prompt | llm

# driver method

In [5]:
def polish_resume(job_description, section, section_content):
    # Generate knowledge for writing effective resume section
    generated_knowledge = generate_knowledge_chain.invoke({"section":section})

    # print(generated_knowledge) # test
    
    # Merge all prompts into one message
    messages = [
        HumanMessage(content=task_overview_prompt.invoke({"section":section}).text),
        HumanMessage(content=input_values_prompt.invoke({"section":section,
                                                         "section_content":section_content,
                                                         "job_description":job_description}).text),
        HumanMessage(content=detail_instruction_prompt.invoke({"section":section, "generated_knowledge":generated_knowledge}).text),
        HumanMessage(content=f"Tailored {section}:")
    ]

    # print(f"prompt:\n{messages}") # test
    
    return tailor_section_chain.invoke({"messages":messages})

# Create UI using Gradio

In [6]:
# Create Gradio interface for the resume polish application, marking polish_prompt as optional
resume_polish_application = gr.Interface(
    fn=polish_resume,
    allow_flagging="never", # Deactivate the flag function in gradio as it is not needed.
    inputs=[
        gr.Textbox(label="Job Description", placeholder="Enter the targeted job description...", lines=20),
        gr.Textbox(label="Resume Section Name", placeholder="Paste your resume section name here..."),
        gr.Textbox(label="Resume Section Content", placeholder="Paste your resume section content here...", lines=20)
    ],
    outputs=gr.Textbox(label="Tailored Resume Section"),
    theme=gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.neutral, neutral_hue=gr.themes.colors.gray),
    title="ResumeRefine AI",
    description="ResumeRefine AI is a state-of-the-art tool that enhances your resume to align perfectly with job descriptions. ResumeRefine AI analyzes and refines your resume content, ensuring it matches the key requirements of your target job. With an intuitive interface built on Gradio, this application streamlines the process of resume customization, giving you a competitive edge in your job search."
)
# Launch the application
resume_polish_application.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
